In [1]:
!pip install ankh --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 42.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
s3fs 2025.3.0 requires fsspec==2025.3.0.*, but you have fsspec 2024.6.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have 

In [2]:
import torch
import ankh
from tqdm.auto import tqdm
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torch.nn.utils.rnn import pad_sequence
import matplotlib.pyplot as plt
import numpy as np
import csv
import math
import pandas as pd
from transformers.models import convbert
from typing import Optional
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
from torch.nn.utils import weight_norm

2025-12-19 16:22:18.981303: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766161339.161287      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766161339.212084      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Available device:', device)

Available device: cuda:0


In [4]:
TCN_LEVEL = 3
NUM_CHANNELS = [256] * TCN_LEVEL
KERNEL_SIZE = 5
NUM_HEADS = 8
DROPOUT = 0.5

In [5]:
class Transformer(nn.Module):
    def __init__(self, input_dim, num_heads, num_layers=1):
        super().__init__()
        encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

    def forward(self, x, padding_mask=None):
        x = x.transpose(0, 1)
        x = self.transformer_encoder(x, src_key_padding_mask=padding_mask)
        x = x.transpose(0, 1)
        return x

class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, dropout):
        super().__init__()
        
        padding = (kernel_size - 1) * dilation

        self.conv1 = nn.Sequential(
            weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size=kernel_size, padding=padding,
                                 stride=stride, dilation=dilation)),
            Chomp1d(padding),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        
        self.conv2 = nn.Sequential(
            weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size=kernel_size, padding=padding,
                                 stride=stride, dilation=dilation)),
            Chomp1d(padding),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(res + out)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size, dropout):
        super().__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers.append(
                TemporalBlock(in_channels, out_channels, kernel_size, stride=1,
                              dilation=dilation_size, dropout=dropout)
            )
        self.network = nn.Sequential(*layers)
        

    def forward(self, x):
        x = self.network(x)
        return x

class BiTCN(nn.Module):
    def __init__(self, input_dim, num_channels, kernel_size, dropout):
        super().__init__()
        self.forward_tcn = TemporalConvNet(input_dim, num_channels, kernel_size, dropout)
        self.backward_tcn = TemporalConvNet(input_dim, num_channels, kernel_size, dropout)

    def forward(self, x):
        forward_out = self.forward_tcn(x)
        backward_out = self.backward_tcn(torch.flip(x, dims=[2]))
        backward_out = torch.flip(backward_out, dims=[2])
        out = torch.cat([forward_out, backward_out], dim=1)
        return out

class PSSPModel(nn.Module):
    def __init__(self, num_channels, kernel_size, num_heads, dropout, input_dim=1536, num_layers=1, num_classes=8):
        super().__init__()
        self.bitcn = BiTCN(input_dim, num_channels, kernel_size, dropout)
        self.transformer = Transformer(num_channels[-1]*2, num_heads, num_layers)
        self.fc = nn.Linear(num_channels[-1]*2, num_classes)

    def forward(self, x, padding_mask=None):
        x = x.transpose(1, 2)
        x = self.bitcn(x)
        x = x.transpose(1, 2)
        x = self.transformer(x, padding_mask)
        logits = self.fc(x)
        return logits

In [6]:
def embed_dataset(model, sequences, shift_left = 0, shift_right = -1):
    inputs_embedding = []
    with torch.no_grad():
        for sample in tqdm(sequences, disable=True):
            ids = tokenizer.batch_encode_plus([sample], add_special_tokens=True, 
                                              padding=True, is_split_into_words=True, 
                                              return_tensors="pt")
            embedding = model(input_ids=ids['input_ids'].to(device))[0]
            embedding = embedding[0].detach().cpu().numpy()[shift_left:shift_right]
            inputs_embedding.append(embedding)
    return inputs_embedding

In [7]:
def preprocess_data(sequences, max_length=None):
    
    sequences = ["".join(seq.split()) for seq in sequences]
    
    if max_length is None:
        max_length = len(max(sequences, key=lambda x: len(x)))

    seqs = [list(seq)[:max_length] for seq in sequences]

    return seqs

In [8]:
def prepare_ankh(device):
    model, tokenizer = ankh.load_large_model()
    model.eval()
    model.to(device=device)

    return model, tokenizer


def prepare_pssp(num_channels, kernel_size, num_heads, dropout, device, 
                 path="/kaggle/input/bitcn-transformer/pytorch/default/3/5_256_3_8.pt"):
    model_pssp = PSSPModel(num_channels=num_channels, 
                  kernel_size=kernel_size,
                  num_heads=num_heads,
                 dropout=dropout).to(device)
    model_pssp.load_state_dict(torch.load(path)['model_state_dict'])

    return model_pssp.eval()
    
def prepare_data(model, sequences, shift_left = 0, shift_right = -1, max_length = None):
    prepared_seq = preprocess_data(sequences, max_length)
    embed_seq = embed_dataset(model, prepared_seq, shift_left, shift_right)

    return embed_seq

    
def predict_pssp(model_pssp, seq, device):
    unique_tags = {'B', 'C', 'E', 'G', 'H', 'I', 'S', 'T'}
    tag2id = {'B': 0, 'C': 1, 'I': 2, 'T': 3, 'S': 4, 'E': 5, 'G': 6, 'H': 7}
    id2tag = {0: 'B', 1: 'C', 2: 'I', 3: 'T', 4: 'S', 5: 'E', 6: 'G', 7: 'H'}

    seq = torch.tensor(seq).to(device)
    
    with torch.no_grad():
        with torch.cuda.amp.autocast():
            outputs = model_pssp(seq)
            
    preds = outputs.argmax(dim=2).cpu().numpy()
    
    pred_seq = []
    for i in range(preds.shape[0]):
        for j in range(preds.shape[1]):
            pred_seq.append(id2tag[preds[i][j]])

    seq_string = "".join(pred_seq)
    
    return seq_string

In [9]:
model, tokenizer = prepare_ankh(device=device)
model_pssp = prepare_pssp(num_channels=NUM_CHANNELS, kernel_size=KERNEL_SIZE, num_heads=NUM_HEADS, dropout=DROPOUT, device=device)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/849 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/7.52G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.51G [00:00<?, ?B/s]

In [10]:
sequence = input("Masukkan sequence asam amino: ")

seq = prepare_data(model=model, sequences=[sequence])
predicted = predict_pssp(model_pssp=model_pssp, seq=seq, device=device)
print("Sequence prediksi struktur sekunder: ", predicted)

Masukkan sequence asam amino:  TTYADFIASGRTGRRNAIHD


Sequence prediksi struktur sekunder:  CCHHHHHHTTCCCCCCCCCC
